In [ ]:
! pip install opencv-python == 4.5.5.64

In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from loader_util.nn.conv import Resnet
from loader_util.callbacks import EpochCheckpoint
from loader_util.callbacks import TrainingMonitor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import numpy as np
import argparse
import sys

In [ ]:
# script constants
checkpoints_dir = r""
model_to_load = r""
start_epoch = 0

In [ ]:
# load the training and testing data, converting the images from
# integers to floats
print("[INFO] loading CIFAR-10 data...")
((trainx, trainy), (testx, testy)) = cifar10.load_data()
trainx = trainx.astype("float")
testx = testx.astype("float")

# apply mean subtraction to the data
mean = np.mean(trainx, axis=0)
trainx -= mean
testx -= mean

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1, horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
# if there is no specific model checkpoint supplied, then initialize
# the network (ResNet-56) and compile the model
if not model_to_load:
    print("[INFO] compiling model...")
    opt = SGD(lr=1e-1)
    model = Resnet.build(32, 32, 3,
                         classes=10,
                         stages=(9, 9, 9),
                         filters=(16, 16, 32, 64),
                         reg=0.0001)
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model_to_load))
    model = load_model(model_to_load)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(
        K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-5)
    print("[INFO] new learning rate: {}".format(
        K.get_value(model.optimizer.lr)))

In [ ]:
# construct the set of callbacks
callbacks = [
    EpochCheckpoint(checkpoints_dir,
                    every=5,
                    startAt=start_epoch),
    TrainingMonitor("output/resnet56_cifar10.png",
                    jsonPath="output/resnet56_cifar10.json",
                    startAt=start_epoch)]

In [ ]:
# train the network
print("[INFO] training network...")
model.fit_generator(
    aug.flow(trainx, trainy, batch_size=128),
    validation_data=(testx, testy),
    steps_per_epoch=len(trainx) // 128, epochs=100,
    callbacks=callbacks, verbose=1)